# Analysing text with ```spaCy```

There are a number of different NLP frameworks that you're likely to encounter. The most popular and widely-used of these are:

- ```NLTK``` (Natural Language Toolkit, old-school)
- ```UDPipe``` (Neural network based, fast and light, but not super accurate)
- ```CoreNLP``` and ```stanza``` (Created by the team at Stanford; academically robust)
- ```spaCy``` production-ready, well-documented, state-of-the-art

We'll be working with ```spaCy``` in this module, primarily because it's easy and intuitive, and also scales well.

First thing we need to do is install ```spaCy``` and the language model that we want to use.

From the command line, you should first make sure to run the setup script to install requirements with `./setup.sh`. Or run these commands yourself:
```bash
pip install spacy pandas
python -m spacy download en_core_web_md
```

## Initializing ```spaCy```

The first thing we need to do is import ```spaCy``` __and__ the language model that we want to use.

Note that, if you want to use different languages you want to use different language models.

In [2]:
# create a spacy NLP class
import spacy
nlp = spacy.load("en_core_web_md")

With the model now loaded, we can begin to do some very simple NLP tasks.

Here, we create a spaCy object and assign it to the variable ```nlp```. This is the NLP pipeline that will do all our heavy lifting, using the trained model we've specified.

Below, you can see what the pipeline does with a bit of sample text. Passing text to the nlp object gives us access to a bunch of properties, including tokens (words), parts of speech, named entities, and so on. Here's we two of them, tokens and entities. These objects, in turn, have certain methods attached to them. A full outline of available methods can be found in the spaCy docs.

In this case, for all token objects, let's return the token itself (```token.text```); its part-of-speech tag (```token.pos_```); and the grammatical dependency relations between the tokens (```token.dep_```).


In [3]:
# a single sentence example
input_string = "My name is Ross and I have family in New York City."

In [4]:
# create a new Doc object
doc = nlp(input_string)

In [5]:
print(doc)

My name is Ross and I have family in New York City.


## Tokens

In [6]:
# tokenizing text
for token in doc:
    print(token.text)

My
name
is
Ross
and
I
have
family
in
New
York
City
.


## Index, text and POS-tag

In [7]:
# find parts-of-speech
for token in doc:
    print(token.i, token.text, token.pos_)

0 My PRON
1 name NOUN
2 is AUX
3 Ross PROPN
4 and CCONJ
5 I PRON
6 have VERB
7 family NOUN
8 in ADP
9 New PROPN
10 York PROPN
11 City PROPN
12 . PUNCT


## NER labels
Extracting named entities from a ```spaCy``` doc requires an extra step, but nothing too challenging:

In [8]:
# extracting NERs
for ent in doc.ents:
    print(ent.text, ent.label_)

Ross PERSON
New York City GPE


## Count distribution of linguistic features

### Create doc object

In [10]:
# load a text file
import os
filename = os.path.join("..", "data", "au-history", "au-history.txt")
with open(filename, "r", encoding="utf-8") as file:
    text = file.read()

In [11]:
nlp = spacy.load("en_core_web_md")
# The text has been formatted (manually) with double line breaks between headers and paragraphs.
# One could also have done this with preprocessing if there had been many more documents!
# With the double line breaks, we tell spacy to split sentences on those.
nlp.add_pipe("sentencizer", config={"punct_chars": ["\n\n"]}, before="parser")

In [13]:
# create a doc object
doc = nlp(text)

In [14]:
for sent in doc.sents:
    print(sent.text)
    print('-' * 50)

History


--------------------------------------------------
AU will soon be celebrating its 100th anniversary. Since its foundation in 1928, the university has evolved from 78 students to approx. 38,000 students today.


--------------------------------------------------
1928
Establishment
After a long, persistent struggle and strong unity among the citizens of Aarhus, university education finally opened its doors.

‘
--------------------------------------------------
University Teaching in Jutland’ started up with 78 students in rented premises, where they were taught French, English, German, Danish and introductory philosophy.


--------------------------------------------------
1933
The first university building is inaugurated
On 11 September 1933, King Christian X inaugurated the first university building. The building was designed by Kay Fisker and C. F. Møller, and its design became the template for the buildings in the University Park.


----------------------------------------

### Entities

In [15]:
# create empty list
entities = []

# add each entity to list
for ent in doc.ents:
    entities.append((ent.text, ent.label_))

In [16]:
print(set(entities))

{('the Nobel Prize in Economic Sciences', 'WORK_OF_ART'), ('The Faculty of Health Sciences', 'ORG'), ('2019', 'DATE'), ('nine', 'CARDINAL'), ('11 September 1933', 'DATE'), ('78', 'CARDINAL'), ('two', 'CARDINAL'), ('the Faculty of Health Sciences', 'ORG'), ('English', 'LANGUAGE'), ('2025', 'DATE'), ('2025', 'CARDINAL'), ('20', 'CARDINAL'), ('four to five', 'CARDINAL'), ('Gestapo', 'ORG'), ('Business and Social Sciences', 'ORG'), ('Dale T. Mortensen Building', 'PERSON'), ('Medicine, Economics, Law', 'ORG'), ('the Faculty of Medical Sciences', 'ORG'), ('Aarhus University’s', 'ORG'), ('Dale T. Mortensen', 'PERSON'), ('1933', 'DATE'), ('C. F. Møller', 'PERSON'), ('the Aarhus University School of Engineering', 'ORG'), ('French', 'LANGUAGE'), ('January 2020', 'DATE'), ('31 October 1944', 'DATE'), ('1977', 'DATE'), ('the International Centre', 'FAC'), ('the Danish University of Education', 'ORG'), ('the Danish Institute of Agricultural Sciences', 'ORG'), ('the Department of Economics and Busin

### Adjective frequency

In [17]:
# count number of adjectives
adjective_count = 0
for token in doc:
    if token.pos_ == "ADJ":
        adjective_count += 1

In [18]:
adjective_count

34

In [19]:
# find the relative frequency per 100 tokens
percent = (adjective_count/len(doc)) * 100

In [20]:
round(percent, 2)

4.27

## Creating neater outputs using ```pandas```

At the moment, all of our output from ```spaCy``` is in the form of lists. If we want to save these, it probably makes sense to have them saved in a more transferable format, such as CSV files or JSONs.

One very easy way to do this with Python is by using the dataframe library ```pandas```.

In [21]:
import pandas as pd

In [22]:
# create spaCy doc
# create a new Doc object
doc = nlp(input_string)

In [23]:
annotations = []
for token in doc:
    annotations.append((token.text, token.pos_))

In [24]:
annotations

[('My', 'PRON'),
 ('name', 'NOUN'),
 ('is', 'AUX'),
 ('Ross', 'PROPN'),
 ('and', 'CCONJ'),
 ('I', 'PRON'),
 ('have', 'VERB'),
 ('family', 'NOUN'),
 ('in', 'ADP'),
 ('New', 'PROPN'),
 ('York', 'PROPN'),
 ('City', 'PROPN'),
 ('.', 'PUNCT')]

In [25]:
# spaCy doc to pandas dataframe
data = pd.DataFrame(annotations, 
                    columns=["Token", "POS"])

In [26]:
data

,Token,POS
0,My,PRON
1,name,NOUN
2,is,AUX
3,Ross,PROPN
4,and,CCONJ
5,I,PRON
6,have,VERB
7,family,NOUN
8,in,ADP
9,New,PROPN


In [27]:
# save dataframe
output_path = os.path.join("..", "output", "annotations.csv")
os.makedirs(os.path.dirname(output_path), exist_ok=True)  # make if it does not exist
data.to_csv(output_path)